In [2]:
import requests
import os
import pandas as pd
import numpy as np

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
url = "https://api.foursquare.com/v3/places/search"
api_key = os.environ['my_key']


headers = {
    "Accept": "application/json",
    "Authorization": api_key  
    
}

In [ ]:
CityByke_locations = pd.read_csv('../data/CityByke.csv')

Latitude = list(CityByke_locations['latitude'])
Longitude = list(CityByke_locations['longitude'])

In [ ]:

name = []
fsq_id =[]
latitude = []
longitude = []
distance = []
address = []
price = []
rating = []

for index in CityByke_locations.index:
    
    params = {
    "fields": "name,fsq_id,categories,rating,location,distance,price,store_id",
    "query": "Restaurants",
    "ll": f"{Latitude[index]},{Longitude[index]}",
    "radius" : 1000,
    # "open_now": "true",
    "sort":"DISTANCE"
    }


    

    response = requests.request("GET", url, params=params, headers=headers)
    response_json = response.json()
    
    for restaurant in response_json['results']:
        
        name.append(restaurant['name'])
        fsq_id.append(restaurant['fsq_id'])
        
        latitude.append(Latitude[index])
        longitude.append(Longitude[index])
        
        distance.append(restaurant['distance'])
        
        try:
            address.append(restaurant['location']['address'])
        except:
            address.append(None)
            
        try:
            price.append(restaurant['price'])
        except:
            price.append(None)
            
        try:
            rating.append(restaurant['rating'])
        except:
            rating.append(None)                      

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
#parsed in the for loop above


Put your parsed results into a DataFrame

I removed the colums to remove the nulls

In [ ]:
restaurant = {"name" : name,
               'fsq_id' : fsq_id,
               # 'category_name' : category_name,
               # 'chains' : chains,
               'latitude' : latitude,
               'longitude' : longitude,
               'distance' : distance,
               'address': address,
               # 'store_id' : store_id,
               # 'locality' : locality,
               'price' : price,
               'rating' : rating
              
              
              }

df_foursquare_restaurants = pd.DataFrame(restaurant)
df_foursquare_restaurants

Continued analysis on this project by Reading the saved result for my foursquare dataframe

In [3]:
df_foursquare_restaurants = pd.read_csv ("../data/foursquare_restaurants.csv")

In [13]:
df_foursquare_restaurants

,latitude,longitude,fsq_restaurant_number,Fsq_avg_distance,Fsq_avg_price,Fsq_avg_rating
0,49.238105,-123.154966,10,362.600000,1.800000,7.866667
1,49.243375,-123.104978,10,317.100000,1.555556,7.983333
2,49.245135,-123.116098,8,671.625000,1.714286,6.775000
3,49.245535,-123.120496,10,535.500000,1.555556,6.640000
4,49.246243,-123.124373,10,362.600000,1.375000,6.100000
...,...,...,...,...,...,...
240,49.295340,-123.150433,2,881.000000,2.000000,8.700000
241,49.297933,-123.130758,10,649.700000,2.000000,8.044444
242,49.298657,-123.121176,6,858.166667,2.400000,7.820000
243,49.299292,-123.130281,10,739.200000,2.000000,8.044444


In [6]:
df_foursquare_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   latitude               245 non-null    float64
 1   longitude              245 non-null    float64
 2   fsq_restaurant_number  245 non-null    int64  
 3   Fsq_avg_distance       245 non-null    float64
 4   Fsq_avg_price          245 non-null    float64
 5   Fsq_avg_rating         245 non-null    float64
dtypes: float64(5), int64(1)
memory usage: 11.6 KB


In [4]:
# Data Check 
fsq_null_count = df_foursquare_restaurants.isnull().sum().sort_values(ascending=False)
fsq_null_percent = (df_foursquare_restaurants.isnull().sum()/df_foursquare_restaurants.isnull().count()).sort_values(ascending=False)
fsq_missing_data = pd.concat([fsq_null_count, fsq_null_percent], axis=1, keys=['Total', 'Percent'])
fsq_missing_data


,Total,Percent
latitude,0,0.0
longitude,0,0.0
fsq_restaurant_number,0,0.0
Fsq_avg_distance,0,0.0
Fsq_avg_price,0,0.0
Fsq_avg_rating,0,0.0


In [11]:
df_foursquare_restaurants.describe()

,latitude,longitude,fsq_restaurant_number,Fsq_avg_distance,Fsq_avg_price,Fsq_avg_rating
count,245.000000,245.000000,245.000000,245.000000,245.000000,245.000000
mean,49.271867,-123.117509,9.844898,374.025739,1.765071,8.042894
std,0.011765,0.030170,0.991989,171.633622,0.276792,0.394994
min,49.238105,-123.209176,2.000000,75.700000,1.100000,6.100000
25%,49.263518,-123.136972,10.000000,243.500000,1.600000,7.860000
50%,49.271377,-123.117772,10.000000,341.500000,1.700000,8.080000
75%,49.280852,-123.097978,10.000000,482.400000,1.900000,8.260000
max,49.302577,-123.035969,10.000000,914.200000,2.666667,8.840000


As part of my data observation above, i noticed that the foursquare API restricted the number results per API call to 10 hence the number of restaurants per location are not more than 10 which is not a true representation of the total number of restaurants close to the bike stations. However the column can be used as a yard stick to know the number of dataset used to derive the result

In [ ]:
# Data Transformation

pointOfInterest = df_foursquare_restaurants.groupby(['latitude', 'longitude']).agg({'fsq_id': 'count', 'distance': np.mean, 'price': np.mean, 'rating':np.mean})
poi_foursquare_restaurants= pointOfInterest.rename(columns= {'fsq_id':'fsq_restaurant_number', 'distance':'Fsq_avg_distance','price': 'Fsq_avg_price', 'rating': 'Fsq_avg_rating' })
poi_foursquare_restaurants.to_csv('../data/foursquare_restaurants.csv')

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
yelp_url = "https://api.yelp.com/v3/businesses/search"
api_key = os.environ['yelp_key']


headers = {
    "Accept": "application/json",
    "Authorization": f'Bearer {api_key}'  
    
}


name = []
id = []
category_name = []
latitude = []
longitude = []
distance = []
address = []
price = []
review_count = []
rating = []


for index in CityByke_locations.index:
    
    
    params = {

    "term": "restaurants",
    "latitude": Latitude[index],
    "longitude": Longitude[index],
    "radius" : 1000,
    # "open_now": "true",
    "sort_by":"distance"
    }
    
    
    response = requests.request("GET", yelp_url, params=params, headers=headers)
    response_json = response.json()
    
    
    for business in response_json['businesses']:
        name.append(business['name'])
        id.append(business['id'])
        # category_name.append(business['categories'][0]['title'])
        latitude.append(Latitude[index])
        longitude.append(Longitude[index])
        address.append(business['location']['display_address'])
        distance.append(business['distance'])
        
        try:
            price.append(business['price'])
        except:
            price.append(None)
            
        review_count.append(business['review_count'])
        rating.append(business['rating'])
            


    


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
#parsed in the for loop above

Put your parsed results into a DataFrame

In [ ]:
restaurant = {"name" : name,
               'id' : id,
               # 'category_name' : category_name,
               # 'chains' : chains,
               'latitude' : latitude,
               'longitude' : longitude,
               'distance' : distance,
               'address': address,
               # 'store_id' : store_id,
               'price' : price,
               'review_count' : review_count,
               'rating' : rating
              
              
              }   

df_yelp_restaurants = pd.DataFrame(restaurant)

#allign the rating structure with foursquare data
df_yelp_restaurants['adj_rating'] =  df_yelp_restaurants['rating'] *2




Ran out of API calls for my API keys, so i will Continue analysis on this project by Reading the saved result for my foursquare dataframe

In [7]:
df_yelp_restaurants = pd.read_csv ("../data/yelp_restaurants.csv")

In [12]:
df_yelp_restaurants

,latitude,longitude,Yelp_restaurant_number,Yelp_avg_distance,Yelp_avg_review_count,Yelp_avg_rating
0,49.238105,-123.154966,20,310.957054,50.550000,6.500000
1,49.243375,-123.104978,20,327.918451,78.500000,6.600000
2,49.245135,-123.116098,20,778.897047,66.550000,6.300000
3,49.245535,-123.120496,20,657.078501,63.500000,6.350000
4,49.246243,-123.124373,20,589.512964,42.600000,6.200000
...,...,...,...,...,...,...
240,49.295340,-123.150433,20,960.011242,64.700000,6.700000
241,49.297933,-123.130758,20,627.016350,152.450000,6.700000
242,49.298657,-123.121176,20,967.824466,99.250000,6.850000
243,49.299292,-123.130281,20,735.486976,152.450000,6.700000


In [8]:
# Data Check and cleaning 
yelp_null_count = df_yelp_restaurants.isnull().sum().sort_values(ascending=False)
yelp_null_percent = (df_yelp_restaurants.isnull().sum()/df_yelp_restaurants.isnull().count()).sort_values(ascending=False)
yelp_missing_data = pd.concat([yelp_null_count, yelp_null_percent], axis=1, keys=['Total', 'Percent'])
yelp_missing_data



,Total,Percent
latitude,0,0.0
longitude,0,0.0
Yelp_restaurant_number,0,0.0
Yelp_avg_distance,0,0.0
Yelp_avg_review_count,0,0.0
Yelp_avg_rating,0,0.0


i had dropped the column ('price' ) during the code development because it had a lot of nulls. It was droped before i saved my csv this is why it is not shown in the table (yelp_missing_data) above. Code to drop was removed with my rough works, hence i have included it below 



In [ ]:
# Drop Price column since its data is not useful
df_yelp_restaurants.drop(columns=['price'], inplace=True)

In [10]:
df_yelp_restaurants.describe()

,latitude,longitude,Yelp_restaurant_number,Yelp_avg_distance,Yelp_avg_review_count,Yelp_avg_rating
count,245.000000,245.000000,245.000000,245.000000,245.000000,245.000000
mean,49.271867,-123.117509,19.865306,262.936946,77.498027,6.535578
std,0.011765,0.030170,1.488419,172.994930,39.613720,0.712376
min,49.238105,-123.209176,3.000000,44.955179,4.850000,3.666667
25%,49.263518,-123.136972,20.000000,132.465398,51.300000,6.100000
50%,49.271377,-123.117772,20.000000,225.874907,68.450000,6.600000
75%,49.280852,-123.097978,20.000000,344.340706,97.400000,7.000000
max,49.302577,-123.035969,20.000000,967.824466,236.350000,8.200000


As part of my data observation above, i noticed that the Yelp API restricted the number results per API call to 20 hence the number of restaurants per location are not more than 20 which is not a true representation of the total number of restaurants close to the bike stations. However the column can be used as a yard stick to know the number of dataset used to derive the result

In [ ]:
# Data Transformation

pointOfInterest = df_yelp_restaurants.groupby(['latitude', 'longitude']).agg({'id': 'count','distance': np.mean , 'review_count': np.mean, 'adj_rating':np.mean})
poi_yelp_restaurants= pointOfInterest.rename(columns= {'id': 'Yelp_restaurant_number', 'distance':'Yelp_avg_distance','review_count' : 'Yelp_avg_review_count', 'adj_rating': 'Yelp_avg_rating' })
poi_yelp_restaurants.to_csv('yelp_restaurants.csv')
poi_yelp_restaurants

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Observing Vancouver, foursquare API seem to have the higher quality of data. I observe it has more attribute. 
For example, foursquare API provide muliple category for each restaurant. so i can observe the different at a glance which restaurant has the most variants of food
It also gives complete information about the price category of each restaurant so i know at a glance how expensive each restaurant is

Get the top 10 restaurants according to their rating

In [ ]:
poi_yelp_restaurants.sort_values('Yelp_avg_rating', ascending=False).head(10)

In [ ]:
poi_foursquare_restaurants.sort_values('fsq_avg_rating', ascending=False).head(10)